https://www.kaggle.com/c/competitive-data-science-predict-future-sales

https://nbviewer.jupyter.org/github/fastai/course-v3/blob/master/nbs/dl1/lesson6-rossmann.ipynb

In [1]:
%reload_ext autoreload
%autoreload 2

In [3]:
#Do the fastai setup on ubenutu before
from fastai.tabular import *

import numpy as np
import pandas as pd
import zipfile
import os

from dfply import *
import plotly_express as px

In [11]:
#Read data from Kaggle
#Train
! wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/8587/sales_train.csv.gz?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1557939488&Signature=T4CagEMATDVrMyQEb8YRPWjvTu%2BZAs2Jpf4g0jqqSsgrMAIndI%2BmMhpBFGLfnMD9ssOUud3Cu4zDl6OpvFQK2Xm5Rdq1KnIZvIojr7Q3lFGJ4%2FvL0e6zgaeucD5a7QZ8tpzV8ZeHL0BzE2s2TN89pzcRKbbOal4dfv2T1Dre3sRM48e4%2FLcc7PpgwIWz2RPXiGBrAXHbLLOCxpur3B%2B0MTm4jXAUpqfDzQ%2FnHdwEyGubcoxRnL%2Bi5H96hgKV20vG860q%2B8ioQUYw3byIibUUGlCbT91NQ2PacEnaE5ncsq5NYFvs9sy1vmXw9VcIs%2BEjlOqvszLdt5rZpHnN96FMLw%3D%3D" -O "sales_train.csv.gz" -c

#test
! wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/8587/test.csv.gz?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1557939625&Signature=STEoYJTfqZpzevTmynvlj9vXvz712eJ8rmKR6z5On9KemM%2BDMn%2Fq13vGVQa%2BUahXA7xgQA9BnlJJg6HgTLDjDVy2o7MpgvpsTTykGfyyCKggODlASiIymQS%2F4tEk942rthvxd7yeH80vaY0xcavqaUXb38EYuitU%2FAxtEgyczW3b3HwT7EBng%2Fe5qRLOHTwFK4Db%2FseUDNmXXFTusazNHdW2SZm2erz34GSCTWI38Y4mqr0c9wVC5l8MvetSNlTACTXY55ZyucO23NRQvs7pozC8LN10GtmdCn3z156SsEzfP59%2B0hLKO202%2BM8Y7pBOTFgS2K5Eo8VzPxbm2ZA2zw%3D%3D" -O "test.csv.gz" -c

#item
! wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/8587/items.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1557940228&Signature=nP6pLqPRS0c7vDYEX8VyhD7Mt7rS3rDgZn2dnclCWeA%2B0u9MQIRd%2BbxsTecV63ERNq3lQDlg1eeWm82kQKNlefxNxqxysISxJdAR2K1lopb68vGF9xkG2RoT%2F2mQxFtZWXpO7lhKvCiy4lhGKmKQIPSLGEcdpOVmVdJqVYAwGruXdQeXIOxa4SkZ2v%2B%2BbLgOow4k69r35%2FabPjrSXzH0BhQ%2BodPuMELGmhu9PFGYscyDpmzmHNLLBKpJ2ChPevkwp%2BZ3FUzSu5nLy7i4O4hFVJHAo8hDN3A1Xu%2Fy3OskfgHu7AOwFvF5c77TR%2B2na8idzPe%2FotNVD0%2BKmQBwlxFRIg%3D%3D" -O "items.csv" -c

#shops
! wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/8587/shops.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1557940264&Signature=Bq864ts42ydZLQdwckpei%2Fvng6YQU6UnPZEsYko1lam1zz5rFolNNOoZtxGSWyEX76Qg06C2KFLkHvOnO4dynR3ZR3y4D8QfYtapsZjLecfccxj2YZ7Zc9eJ6xJ7QX4EhEOlcyXgI3fjbtGNysyslhFRSrStQ2CqDW1lobLsO49kM6DRnmd%2BK0ZWEnVThVNa%2Fo51ZXiBlSIPO4dJLcjTOKzfB4oYmZXoK8ROjkDp3RRbbja78k9bKrFf2%2BlutCjcNrHUEmqe6VDexXrakikgg4vQF3vX1V5arj1hu19vdaNkQEAPlVnYHzc3Gka4kNXAZ8uHeCd1F06prQOBWvltoQ%3D%3D" -O "shops.csv" -c



--2019-05-12 17:11:40--  https://storage.googleapis.com/kaggle-competitions-data/kaggle/8587/sales_train.csv.gz?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1557939488&Signature=T4CagEMATDVrMyQEb8YRPWjvTu%2BZAs2Jpf4g0jqqSsgrMAIndI%2BmMhpBFGLfnMD9ssOUud3Cu4zDl6OpvFQK2Xm5Rdq1KnIZvIojr7Q3lFGJ4%2FvL0e6zgaeucD5a7QZ8tpzV8ZeHL0BzE2s2TN89pzcRKbbOal4dfv2T1Dre3sRM48e4%2FLcc7PpgwIWz2RPXiGBrAXHbLLOCxpur3B%2B0MTm4jXAUpqfDzQ%2FnHdwEyGubcoxRnL%2Bi5H96hgKV20vG860q%2B8ioQUYw3byIibUUGlCbT91NQ2PacEnaE5ncsq5NYFvs9sy1vmXw9VcIs%2BEjlOqvszLdt5rZpHnN96FMLw%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.6.80, 2607:f8b0:400a:808::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.6.80|:443... connected.
HTTP request sent, awaiting response... 416 Requested range not satisfiable

    The file is already fully retrieved; nothing to do.

--2019-05-12 17:11:41--  https://storage.googleapis.com/kaggle-competitions-data/kaggle/8587/test.csv.

In [12]:
#Unzip
raw_train = pd.read_csv('sales_train.csv.gz', compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False)
test = pd.read_csv('test.csv.gz', compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False)
shops = pd.read_csv('shops.csv')
items = pd.read_csv('items.csv')

In [15]:
print("train:", raw_train.shape)
print("test:", test.shape)
print("item:", items.shape)
print("shop:", shops.shape)

train: (2935849, 6)
test: (214200, 3)
item: (22170, 3)
shop: (60, 2)


In [16]:
raw_train.head().T

,0,1,2,3,4
date,02.01.2013,03.01.2013,05.01.2013,06.01.2013,15.01.2013
date_block_num,0,0,0,0,0
shop_id,59,25,25,25,25
item_id,22154,2552,2552,2554,2555
item_price,999,899,899,1709.05,1099
item_cnt_day,1,1,-1,1,1


In [17]:
test.head().T

,0,1,2,3,4
ID,0,1,2,3,4
shop_id,5,5,5,5,5
item_id,5037,5320,5233,5232,5268


In [18]:
n = len(raw_train); n

2935849

In [19]:
#Sampling
idx = np.random.permutation(range(n))[:2000]
idx.sort()
small_train_df = raw_train.iloc[idx[:1000]]
small_test_df = raw_train.iloc[idx[1000:]]

In [22]:
raw_train.dtypes

date               object
date_block_num      int64
shop_id             int64
item_id             int64
item_price        float64
item_cnt_day      float64
dtype: object

In [24]:
add_datepart(small_train_df, "date", drop=False)
add_datepart(small_test_df, "date", drop=False)

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/fastai/tabular/transform.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/fastai/tabular/transform.py:63: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/fastai/tabular/transform.py:64: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
1461013,2014-03-30,14,53,7860,399.00,1.0,2014,3,13,30,6,89,False,False,False,False,False,False,1396137600
1462858,2014-03-30,14,52,5816,1199.00,1.0,2014,3,13,30,6,89,False,False,False,False,False,False,1396137600
1467336,2014-03-16,14,46,14054,399.00,1.0,2014,3,11,16,6,75,False,False,False,False,False,False,1394928000
1467506,2014-03-15,14,42,3943,1299.00,1.0,2014,3,11,15,5,74,False,False,False,False,False,False,1394841600
1468529,2014-03-01,14,47,6490,699.00,1.0,2014,3,9,1,5,60,False,True,False,False,False,False,1393632000
1468871,2014-03-25,14,47,6692,4599.00,1.0,2014,3,13,25,1,84,False,False,False,False,False,False,1395705600
1469675,2014-03-16,14,47,20610,799.00,2.0,2014,3,11,16,6,75,False,False,False,False,False,False,1394928000
1470033,2014-03-30,14,47,14169,174.00,1.0,2014,3,13,30,6,89,False,False,False,False,False,False,1396137600
1470864,2014-03-21,14,43,6691,2199.00,1.0,2014,3,12,21,4,80,False,False,False,False,False,False,1395360000
1474399,2014-03-23,14,57,4351,399.00,1.0,2014,3,12,23,6,82,False,False,False,False,False,False,1395532800


In [26]:
small_train_df.head().T

,1609,4370,5234,9957,10223
date,2013-05-01 00:00:00,2013-01-17 00:00:00,2013-01-14 00:00:00,2013-01-20 00:00:00,2013-01-23 00:00:00
date_block_num,0,0,0,0,0
shop_id,25,25,25,19,19
item_id,3556,12565,16227,1481,1921
item_price,2198.5,199,379,1999,249
item_cnt_day,1,1,1,1,1
Year,2013,2013,2013,2013,2013
Month,5,1,1,1,1
Week,18,3,3,3,4
Day,1,17,14,20,23


In [27]:
small_cont_vars = ['item_price','date_block_num','Week']
small_cat_vars =  ['item_id', 'shop_id','Dayofweek']

In [28]:
small_train_df = small_train_df[small_cat_vars + small_cont_vars + ['item_cnt_day']]
small_test_df = small_test_df[small_cat_vars + small_cont_vars + ['item_cnt_day']]

In [31]:
def null_percent_original(series):
    return (pd.isnull(series).sum()/series.size) * 100

@make_symbolic
def null_percent(series):
    return null_percent_original(series)

#Just for display of DF on Notebook
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)
@make_symbolic
def np_where(bools, val_if_true, val_if_false):
    return np.where(bools, val_if_true, val_if_false)

@make_symbolic
def qcut(series, n):
    return pd.qcut(series, n)

In [32]:
#Missing
for k in raw_train.columns:
    display_all(raw_train >> summarise_each([null_percent],k))

,date_wrapper
0,0.0


,date_block_num_wrapper
0,0.0


,shop_id_wrapper
0,0.0


,item_id_wrapper
0,0.0


,item_price_wrapper
0,0.0


,item_cnt_day_wrapper
0,0.0
